In [2]:
import numpy as np
import pandas as pd
import random as ra
from collections import OrderedDict
from datetime import datetime
import time
import warnings
import threading

In [3]:
import itchat
from WindPy import w

In [4]:
def gen_stock2index(windcodes):
    
    stock2index = {}
    for counter, ticker in enumerate(windcodes.split(',')):
        stock2index[ticker] = counter
        
    return stock2index

In [5]:
def gen_index2stock(windcodes):
    
    index2stock = {}
    for counter, ticker in enumerate(windcodes.split(',')):
        index2stock[counter] = ticker
    
    return index2stock

In [6]:
def get_lastclose(windcodes, fields="close"):

    stock = np.array(w.wsd(codes=windcodes, fields="close").Data[0])
    stock[1::2] = 0
    
    cvbonds = np.array(w.wsd(codes=windcodes, fields="close",Priceadj="DP").Data[0])
    cvbonds[::2] = 0
    
    return stock + cvbonds

In [7]:
def get_rtlast(tickers):
    output = w.wsq(codes=tickers, fields="rt_last")
    return np.array(output.Data[0])

In [8]:
def gen_alert(lastclose, rt_close, threshold=0.03):
    
    rt_close[rt_close == 0] = 'nan'
    
    ratio = rt_close/lastclose
    diff = ratio[1::2] - ratio[::2]
    
    states = (diff > threshold) + (diff < -threshold)
    
    #ratio = rt_close/lastclose
    
    return states, ratio

In [9]:
stock2cvbond = {"002916.SZ":"128088.SZ","600105.SH":"110058.SH","300168.SZ":"123021.SZ",
"002391.SZ":"128055.SZ","601211.SH":"113013.SH","000876.SZ":"127015.SZ","603019.SH":"113517.SH",
"002002.SZ":"128085.SZ","603636.SH":"113540.SH","000070.SZ":"127008.SZ","600831.SH":"733831.SH",
"300059.SZ":"123006.SZ","002368.SZ":"128078.SZ","603738.SH":"113503.SH","300545.SZ":"123038.SZ",
"000783.SZ":"127005.SZ","601878.SH":"113022.SH","600567.SH":"110047.SH","000665.SZ":"127007.SZ",
"002460.SZ":"128028.SZ","300088.SZ":"123022.SZ","603688.SH":"113548.SH","600728.SH":"110050.SH",
"000700.SZ":"127004.SZ","601985.SH":"113026.SH","002131.SZ":"128038.SZ","601137.SH":"113031.SH",
"603876.SH":"113534.SH","002455.SZ":"128093.SZ","300497.SZ":"123020.SZ","002497.SZ":"128065.SZ",
"300655.SZ":"123031.SZ","600233.SH":"110046.SH","603305.SH":"113522.SH","300003.SZ":"123040.SZ",
"603626.SH":"113521.SH","603901.SH":"113559.SH","002352.SZ":"128080.SZ","300618.SZ":"123017.SZ",
"001965.SZ":"127012.SZ","300568.SZ":"123009.SZ"}

In [10]:
order_stock2cvbond = OrderedDict(stock2cvbond)

In [11]:
windcodes = str(stock2cvbond).replace('\'','').replace(':',',').replace('{','').replace('}','').replace(' ','')

In [12]:
stock2index = gen_stock2index(windcodes)
index2stock = gen_index2stock(windcodes)

In [13]:
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [16]:
lastclose = get_lastclose(windcodes)

In [18]:
rt_close = get_rtlast(windcodes)

In [19]:
rt_close

array([224.88 , 161.19 ,   5.16 , 115.9  ,  22.02 , 161.25 ,   9.89 ,
       136.825,  17.76 , 122.99 ,  29.29 , 150.95 ,  44.16 , 122.76 ,
         4.4  , 119.061,  13.75 , 135.71 ,  11.02 , 206.3  ,   9.34 ,
         0.   ,  17.89 ,   0.   ,  44.36 , 161.79 ,  27.69 , 165.87 ,
        34.57 , 135.3  ,   7.02 , 122.62 ,  11.66 , 119.76 ,   3.42 ,
       120.84 ,   6.6  , 125.297,  50.04 , 128.568,  10.34 , 178.2  ,
        27.28 , 166.76 ,  10.01 , 134.   ,  13.05 , 179.96 ,   4.7  ,
       107.32 ,   4.35 , 254.593,  13.28 , 131.18 ,  16.52 , 124.31 ,
         5.53 , 115.907,  26.89 , 181.478,   9.19 , 122.582,  39.18 ,
       213.11 ,  13.59 , 126.27 ,  41.83 ,   0.   ,  39.58 , 146.487,
        12.65 , 145.5  ,  10.23 , 118.11 ,  47.51 , 138.392,  68.21 ,
       137.603,   7.58 , 112.803,  38.72 , 157.02 ])

In [21]:
def send_msg():
    try:   
        #w.wsq(windcodes, fields="rt_last", func=myCallback)
        rt_close = get_rtlast(windcodes)
        
        states, ratio = gen_alert(lastclose, rt_close)
        
        t = '%s update' % str(datetime.now())
        print(t)
        itchat.send(t,toUserName)
        
        if sum(states) > 0:
            
            msg = ""
            index_alert = np.where(states)

            for i in index_alert[0]:

                # del stock from dict
                if index2stock[i*2] in stock2index:

                    del stock2index[index2stock[i*2]]
                    rt_close[i*2] = 0

                if index2stock[i*2+1] in stock2index:

                    del stock2index[index2stock[i*2+1]]
                    rt_close[i*2+1] = 0

                stock, cvbonds = list(order_stock2cvbond.items())[i]

                stock_chg = "%.2f%%" % ((ratio[2*i]-1)*100) 
                cvbonds_chg = "%.2f%%" % ((ratio[2*i+1]-1)*100) 
                diff = "%.2f%%;\n" % ((ratio[2*i]-ratio[2*i+1])*100)
                
                #msg = ('pay attention to:%s and %s,' % (stock, cvbonds)) 
                #msg2 = ('stock %s change is: %s,' % (stock, stock_chg))
                #msg3 = ('cvbonds %s change is: %s,' % (cvbonds, cvbonds_chg))
                
                msg += stock + '(' + stock_chg + '),' + cvbonds + '(' + cvbonds_chg + '),' + 'diff:' + diff 

            msg2 = "cvbonds up ratio:%.2f%%" % (sum(ratio[1::2] > 1)/len(ratio[1::2])*100) 
            print(msg + msg2)
            itchat.send(msg + msg2, toUserName)
            
        #time.sleep(120)
            
    except KeyboardInterrupt:
        
        itchat.send('Stock_WeChat 已执行完毕！\n',toUserName)
    
    global timer
    timer = threading.Timer(300, send_msg)
    timer.start()

In [19]:
itchat.auto_login(hotReload=True)

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as Jerry


In [23]:
users=itchat.search_chatrooms(name=u'cvbonds')

In [24]:
toUserName= users[0]['UserName']

In [25]:
toUserName

'@@b8496be050681beab8aade80fc9857a33053cf96b3364a9896eda71765297c60'

In [26]:
timer = threading.Timer(1, send_msg)
timer.start()

c:\apps\miniconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater
  
c:\apps\miniconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less
  


2020-03-09 13:08:07.322880 update


c:\apps\miniconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in greater


002368.SZ(-6.92%),128078.SZ(-1.95%),diff:-4.97%;
603738.SH(-3.27%),113503.SH(-0.23%),diff:-3.04%;
300545.SZ(-6.81%),123038.SZ(-3.23%),diff:-3.58%;
600567.SH(-5.54%),110047.SH(-1.77%),diff:-3.77%;
300088.SZ(-6.77%),123022.SZ(-2.95%),diff:-3.82%;
000700.SZ(-4.54%),127004.SZ(2.50%),diff:-7.03%;
002455.SZ(-3.50%),128093.SZ(-0.48%),diff:-3.02%;
300568.SZ(10.00%),123009.SZ(19.46%),diff:-9.46%;
cvbonds up ratio:17.07%


In [22]:
itchat.login()

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as Jerry


In [27]:
itchat.run(True)

Start auto replying.
No uins in 51 message

No uins in 51 message

No uins in 51 message

chatroom member fetch failed with @8ea1456c2a60703994edee59d7895688
No uins in 51 message

No uins in 51 message

No uins in 51 message

No uins in 51 message

No uins in 51 message

No uins in 51 message

No uins in 51 message

chatroom member fetch failed with @8ea1456c2a60703994edee59d7895688
No uins in 51 message

No uins in 51 message

c:\apps\miniconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater
  
c:\apps\miniconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less
  
Request to send a text message to @@b8496be050681beab8aade80fc9857a33053cf96b3364a9896eda71765297c60: 2020-03-09 13:13:08.456621 update


2020-03-09 13:13:08.456621 update


c:\apps\miniconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in greater
Request to send a text message to @@b8496be050681beab8aade80fc9857a33053cf96b3364a9896eda71765297c60: 000070.SZ(0.55%),127008.SZ(3.64%),diff:-3.09%;
002368.SZ(-7.23%),128078.SZ(-2.06%),diff:-5.18%;
603738.SH(-3.03%),113503.SH(0.22%),diff:-3.25%;
300545.SZ(-6.46%),123038.SZ(-3.44%),diff:-3.02%;
600567.SH(-5.26%),110047.SH(-1.88%),diff:-3.39%;
000700.SZ(-4.02%),127004.SZ(3.36%),diff:-7.38%;
300568.SZ(10.00%),123009.SZ(16.76%),diff:-6.76%;
cvbonds up ratio:17.07%


000070.SZ(0.55%),127008.SZ(3.64%),diff:-3.09%;
002368.SZ(-7.23%),128078.SZ(-2.06%),diff:-5.18%;
603738.SH(-3.03%),113503.SH(0.22%),diff:-3.25%;
300545.SZ(-6.46%),123038.SZ(-3.44%),diff:-3.02%;
600567.SH(-5.26%),110047.SH(-1.88%),diff:-3.39%;
000700.SZ(-4.02%),127004.SZ(3.36%),diff:-7.38%;
300568.SZ(10.00%),123009.SZ(16.76%),diff:-6.76%;
cvbonds up ratio:17.07%


chatroom member fetch failed with @8ea1456c2a60703994edee59d7895688
No uins in 51 message

No uins in 51 message

Request to send a text message to @@b8496be050681beab8aade80fc9857a33053cf96b3364a9896eda71765297c60: 2020-03-09 13:18:12.858419 update


...

No uins in 51 message

Request to send a text message to @@b8496be050681beab8aade80fc9857a33053cf96b3364a9896eda71765297c60: 2020-03-09 14:59:30.244907 update


2020-03-09 14:59:30.244907 update


Request to send a text message to @@b8496be050681beab8aade80fc9857a33053cf96b3364a9896eda71765297c60: 002368.SZ(-6.67%),128078.SZ(-2.67%),diff:-4.00%;
300545.SZ(-9.75%),123038.SZ(-3.56%),diff:-6.19%;
600567.SH(-5.54%),110047.SH(-2.37%),diff:-3.17%;
000665.SZ(-8.29%),127007.SZ(-4.07%),diff:-4.22%;
603688.SH(0.40%),113548.SH(-2.93%),diff:3.33%;
603901.SH(6.23%),113559.SH(0.38%),diff:5.85%;
300568.SZ(10.00%),123009.SZ(19.83%),diff:-9.83%;
cvbonds up ratio:4.88%


002368.SZ(-6.67%),128078.SZ(-2.67%),diff:-4.00%;
300545.SZ(-9.75%),123038.SZ(-3.56%),diff:-6.19%;
600567.SH(-5.54%),110047.SH(-2.37%),diff:-3.17%;
000665.SZ(-8.29%),127007.SZ(-4.07%),diff:-4.22%;
603688.SH(0.40%),113548.SH(-2.93%),diff:3.33%;
603901.SH(6.23%),113559.SH(0.38%),diff:5.85%;
300568.SZ(10.00%),123009.SZ(19.83%),diff:-9.83%;
cvbonds up ratio:4.88%


chatroom member fetch failed with @8ea1456c2a60703994edee59d7895688
No uins in 51 message

itchat received an ^C and exit.
Bye~


In [28]:
itchat.logout()

<ItchatReturnValue: {'BaseResponse': {'ErrMsg': '请求成功', 'Ret': 0, 'RawMsg': 'logout successfully.'}}>

LOG OUT!
c:\apps\miniconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater
  
c:\apps\miniconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less
  
Request to send a text message to @@b8496be050681beab8aade80fc9857a33053cf96b3364a9896eda71765297c60: 2020-03-09 15:04:40.549986 update


2020-03-09 15:04:40.549986 update


c:\apps\miniconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in greater
Request to send a text message to @@b8496be050681beab8aade80fc9857a33053cf96b3364a9896eda71765297c60: 002368.SZ(-6.88%),128078.SZ(-2.73%),diff:-4.15%;
300545.SZ(-9.72%),123038.SZ(-3.34%),diff:-6.38%;
600567.SH(-5.82%),110047.SH(-2.37%),diff:-3.44%;
000665.SZ(-7.86%),127007.SZ(-4.07%),diff:-3.79%;
002455.SZ(-3.68%),128093.SZ(-0.49%),diff:-3.19%;
603901.SH(6.33%),113559.SH(0.47%),diff:5.86%;
300568.SZ(10.00%),123009.SZ(20.21%),diff:-10.21%;
cvbonds up ratio:4.88%


002368.SZ(-6.88%),128078.SZ(-2.73%),diff:-4.15%;
300545.SZ(-9.72%),123038.SZ(-3.34%),diff:-6.38%;
600567.SH(-5.82%),110047.SH(-2.37%),diff:-3.44%;
000665.SZ(-7.86%),127007.SZ(-4.07%),diff:-3.79%;
002455.SZ(-3.68%),128093.SZ(-0.49%),diff:-3.19%;
603901.SH(6.33%),113559.SH(0.47%),diff:5.86%;
300568.SZ(10.00%),123009.SZ(20.21%),diff:-10.21%;
cvbonds up ratio:4.88%


...

Request to send a text message to @@b8496be050681beab8aade80fc9857a33053cf96b3364a9896eda71765297c60: 002368.SZ(-6.88%),128078.SZ(-2.73%),diff:-4.15%;
300545.SZ(-9.72%),123038.SZ(-3.34%),diff:-6.38%;
600567.SH(-5.82%),110047.SH(-2.37%),diff:-3.44%;
000665.SZ(-7.86%),127007.SZ(-4.07%),diff:-3.79%;
002455.SZ(-3.68%),128093.SZ(-0.49%),diff:-3.19%;
603901.SH(6.33%),113559.SH(0.47%),diff:5.86%;
300568.SZ(10.00%),123009.SZ(20.21%),diff:-10.21%;
cvbonds up ratio:4.88%


002368.SZ(-6.88%),128078.SZ(-2.73%),diff:-4.15%;
300545.SZ(-9.72%),123038.SZ(-3.34%),diff:-6.38%;
600567.SH(-5.82%),110047.SH(-2.37%),diff:-3.44%;
000665.SZ(-7.86%),127007.SZ(-4.07%),diff:-3.79%;
002455.SZ(-3.68%),128093.SZ(-0.49%),diff:-3.19%;
603901.SH(6.33%),113559.SH(0.47%),diff:5.86%;
300568.SZ(10.00%),123009.SZ(20.21%),diff:-10.21%;
cvbonds up ratio:4.88%


Request to send a text message to @@b8496be050681beab8aade80fc9857a33053cf96b3364a9896eda71765297c60: 2020-03-09 16:26:11.886483 update


2020-03-09 16:26:11.886483 update


Exception in thread Thread-46:
Traceback (most recent call last):
  File "c:\apps\miniconda3\lib\site-packages\urllib3\connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "c:\apps\miniconda3\lib\site-packages\urllib3\util\connection.py", line 80, in create_connection
    raise err
  File "c:\apps\miniconda3\lib\site-packages\urllib3\util\connection.py", line 70, in create_connection
    sock.connect(sa)
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\apps\miniconda3\lib\site-packages\urllib3\connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "c:\apps\miniconda3\lib\site-packages\urllib3\connectionpool.py", line 343, in _make_request
    se

In [34]:
w.stop()